In [1]:
from pymongo import MongoClient
import folium 
import pandas as pd


In [2]:
client = MongoClient('mongodb://localhost:27017/')

db=client.proyecto2

In [3]:
datab = db.proyecto_coll2.find()
df = pd.DataFrame(datab)
display(df) # DataFrame original
df.shape # Comprobar filas y columnas del DF original

,_id,category_code,country_code,deadpooled_year,founded_year,lat,lng,name,number_of_employees,oficina_principal,total_money_raised
0,5d26fe94ded049a943505ad7,Empresa tecnologica,USA,None,2007,34.017606,-118.487267,Mahalo,40,"{'type': 'Point', 'coordinates': [-118.487267,...",$21M
1,5d26fe94ded049a943505ad8,Empresa tecnologica,USA,None,2006,34.090368,-118.393064,Geni,18,"{'type': 'Point', 'coordinates': [-118.393064,...",$16.5M
2,5d26fe94ded049a943505ad9,Empresa tecnologica,USA,None,2002,37.387845,-122.055197,Plaxo,50,"{'type': 'Point', 'coordinates': [-122.055197,...",$28.3M
3,5d26fe94ded049a943505ada,Empresa de videojuegos,USA,None,2006,37.788482,-122.409173,Kyte,40,"{'type': 'Point', 'coordinates': [-122.409173,...",$23.4M
4,5d26fe94ded049a943505adb,Empresa tecnologica,USA,None,2005,37.480999,-122.173887,Jingle Networks,35,"{'type': 'Point', 'coordinates': [-122.173887,...",$88.7M
5,5d26fe94ded049a943505adc,Empresa tecnologica,USA,None,2002,40.726040,-73.995722,Meetup,75,"{'type': 'Point', 'coordinates': [-73.995722, ...",$18.3M
6,5d26fe94ded049a943505add,Empresa tecnologica,USA,None,1984,37.408802,-121.953770,Cisco,63000,"{'type': 'Point', 'coordinates': [-121.95377, ...",$2.5M
7,5d26fe94ded049a943505ade,Empresa tecnologica,USA,None,1994,37.418531,-122.025485,Yahoo!,13600,"{'type': 'Point', 'coordinates': [-122.025485,...",$6.8M
8,5d26fe94ded049a943505adf,Empresa tecnologica,USA,None,1995,37.295005,-121.930035,eBay,15000,"{'type': 'Point', 'coordinates': [-121.930035,...",$6.7M
9,5d26fe94ded049a943505ae0,Empresa tecnologica,USA,None,2005,37.423390,-122.089951,Jajah,110,"{'type': 'Point', 'coordinates': [-122.0899512...",$33M


(848, 11)

In [4]:
db.proyecto_coll2.find()
# Empresas con una o más oficinas

In [5]:
# Se extrae una lista de geolocations
lista_find=[]
for i in range(len(df)):
    lista_find.append(df.oficina_principal[i])       

In [6]:
def findNear(lista_find, radio_max_meters):
# Funcion que busca las empresas cercanas en 'x' metros a partir de las coordenadas de una empresa
        geopoint=lista_find
        return list(db.proyecto_coll2.find({
        "oficina_principal": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    }))

radio_max_meters = 1000 

# Iteración para cada empresa con sus coordenadas
oficinas_cercanas = []
for i in range(len(df)):
    num_offices = findNear(lista_find[i], radio_max_meters)
    oficinas_cercanas.append(len(num_offices))

# Se agrega una nueva columna al DF que muestra el número de empresas en un radio de 500m
df["Prox_ofis"] = oficinas_cercanas

In [7]:
def suma_empleados_cerca(df):
    cnt=0
    for empresa in df:
        cnt+=empresa['number_of_employees']
    return cnt

In [8]:
col_sum_emp=[]
for i, e in df.iterrows():
    empresas_radio = findNear(lista_find[i], radio_max_meters)
    empl = suma_empleados_cerca(empresas_radio)
    col_sum_emp.append(empl)

In [9]:
df["Suma_empl"] = col_sum_emp

In [10]:
df.drop_duplicates(subset='name', keep='first', inplace=True)

In [11]:
df = df[df.Prox_ofis > 5]


In [12]:
df["country_code"].value_counts()

USA    170
Name: country_code, dtype: int64

In [13]:
import numpy as np
df["ranking"] = df["Suma_empl"]*0.03 + df["Prox_ofis"]*0.01 + np.where(df['category_code'] == 'Empresa de videojuegos', 30, 0)

In [14]:
# Se ordena el DF por más numero de oficinas próximas (500m) y por más numero de empleados
df.sort_values(['ranking'], inplace = True, ascending=[False])
df.reset_index()
df_final=df.iloc[0:10, 1:]

In [15]:
df_final

,category_code,country_code,deadpooled_year,founded_year,lat,lng,name,number_of_employees,oficina_principal,total_money_raised,Prox_ofis,Suma_empl,ranking
11,Empresa de videojuegos,USA,None,2007,40.726155,-73.995625,Livestream,120,"{'type': 'Point', 'coordinates': [-73.995625, ...",$14.7M,10,8617,288.61
251,Empresa de videojuegos,USA,None,2005,40.739930,-73.993049,Cellufun,30,"{'type': 'Point', 'coordinates': [-73.9930489,...",$5.23M,13,8490,284.83
399,Empresa de videojuegos,USA,None,2000,40.738476,-73.998726,VISUALPLANT,6,"{'type': 'Point', 'coordinates': [-73.998726, ...",$500k,9,8212,276.45
9,Empresa tecnologica,USA,None,2005,37.423390,-122.089951,Jajah,110,"{'type': 'Point', 'coordinates': [-122.0899512...",$33M,8,9081,272.51
5,Empresa tecnologica,USA,None,2002,40.726040,-73.995722,Meetup,75,"{'type': 'Point', 'coordinates': [-73.995722, ...",$18.3M,10,8617,258.61
693,Empresa tecnologica,USA,None,2007,40.738567,-73.987199,SpaBooker,42,"{'type': 'Point', 'coordinates': [-73.987199, ...",$14.7M,14,8499,255.11
504,Empresa tecnologica,USA,None,2007,40.733928,-73.991334,Buzzd,9,"{'type': 'Point', 'coordinates': [-73.991334, ...",$4M,10,8430,253.00
54,Empresa tecnologica,USA,None,1985,40.731132,-73.991931,AOL,8000,"{'type': 'Point', 'coordinates': [-73.9919311,...",$1B,7,8282,248.53
40,Empresa de videojuegos,USA,None,2006,37.787092,-122.399972,Curse,58,"{'type': 'Point', 'coordinates': [-122.399972,...",$12M,50,3708,141.74
112,Empresa de videojuegos,USA,None,0,37.783898,-122.395234,Rupture,25,"{'type': 'Point', 'coordinates': [-122.395234,...",$3M,49,3624,139.21


In [16]:
# mark each station as a point
m = folium.Map(location=[37.787092, -122.399972], width=900, height=500, zoom_start=2,)
for index, row in df_final.iterrows():
    folium.CircleMarker([row['lat'], row['lng']],
                        radius=5,
                        popup="Name: {} Categoría: {}".format(row['name'],row["category_code"]),
                        fill_color="#3db7e4", # divvy color
                       ).add_to(m)


In [17]:
m

In [18]:
lista_lats = df_final['lat'].tolist()

In [19]:
lista_longs = df_final['lng'].tolist()



In [20]:
'''from pandas.io.json import json_normalize


ydataframe = json_normalize(data = mongodata, record_path = "offices", 
                             meta = ["category_code", "founded_year", "name",
                                    "total_money_raised", "number_of_employees"])
'''

'from pandas.io.json import json_normalize\n\n\nydataframe = json_normalize(data = mongodata, record_path = "offices", \n                             meta = ["category_code", "founded_year", "name",\n                                    "total_money_raised", "number_of_employees"])\n'

In [21]:
import requests
from dotenv import load_dotenv
import os
load_dotenv()

google_k = os.environ["token_maps"]
#res = requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&type=restaurant&keyword=cruise&key={}'.format(lat_api, lng_api, radio_max_meters, google_k)).json()
#res
#["https://URL/v1/orders/{id}".format(id=id) for id in df.id]


In [23]:
place_club="night_club"
place_bar="bar"
place_takeaway="meal_takeaway"

lista_clubs=[]
lista_bar=[]
lista_takeaway=[]

for i in range(len(lista_lats)):
    lista_clubs.append(requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&type={}&key={}'.format(lista_lats[i], lista_longs[i], radio_max_meters, place_club, google_k)).json())
for i in range(len(lista_lats)):
    lista_bar.append(requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&type={}&key={}'.format(lista_lats[i], lista_longs[i], radio_max_meters, place_bar, google_k)).json())
for i in range(len(lista_lats)):
    lista_takeaway.append(requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&type={}&key={}'.format(lista_lats[i], lista_longs[i], radio_max_meters, place_takeaway, google_k)).json())



In [68]:
lista_clubs[0]['results'][5]['name']

"Joe's Pub"

In [56]:
lista_takeaway[0]['results'][1]['name']

"Mighty Quinn's Barbeque"

In [85]:
lista_clubs[9]['results'][11]['name']

'The View Lounge'

In [77]:
cnt=0
for i in range(len(lista_clubs)):
    cnt+=1
    lista_clubs[i]['results'][cnt]

IndexError: list index out of range

In [28]:
len(lista_clubs[4])#['results'][6]['name'])

4

In [38]:
suma_clubs=[]
for i in range(10):
    suma_clubs.append(len(lista_clubs[i]))

In [39]:
df_final['Total_clubs']=suma_clubs

In [41]:
len(lista_bar)

10

In [49]:
len(lista_takeaway[0])

4

In [42]:
suma_bar=[]
for i in range(10):
    suma_bar.append(len(lista_bar[i]))

In [45]:
df_final['Total_bar']=suma_bar

In [43]:
len(lista_takeaway)

10

In [44]:
suma_takeaway=[]
for i in range(10):
    suma_takeaway.append(len(lista_takeaway[i]))

In [46]:
df_final['Total_takeaway']=suma_takeaway

In [47]:
df_final

,category_code,country_code,deadpooled_year,founded_year,lat,lng,name,number_of_employees,oficina_principal,total_money_raised,Prox_ofis,Suma_empl,ranking,Total_clubs,Total_bar,Total_takeaway
11,Empresa de videojuegos,USA,None,2007,40.726155,-73.995625,Livestream,120,"{'type': 'Point', 'coordinates': [-73.995625, ...",$14.7M,10,8617,288.61,4,4,4
251,Empresa de videojuegos,USA,None,2005,40.739930,-73.993049,Cellufun,30,"{'type': 'Point', 'coordinates': [-73.9930489,...",$5.23M,13,8490,284.83,4,4,4
399,Empresa de videojuegos,USA,None,2000,40.738476,-73.998726,VISUALPLANT,6,"{'type': 'Point', 'coordinates': [-73.998726, ...",$500k,9,8212,276.45,4,4,4
9,Empresa tecnologica,USA,None,2005,37.423390,-122.089951,Jajah,110,"{'type': 'Point', 'coordinates': [-122.0899512...",$33M,8,9081,272.51,3,3,3
5,Empresa tecnologica,USA,None,2002,40.726040,-73.995722,Meetup,75,"{'type': 'Point', 'coordinates': [-73.995722, ...",$18.3M,10,8617,258.61,4,4,4
693,Empresa tecnologica,USA,None,2007,40.738567,-73.987199,SpaBooker,42,"{'type': 'Point', 'coordinates': [-73.987199, ...",$14.7M,14,8499,255.11,4,4,4
504,Empresa tecnologica,USA,None,2007,40.733928,-73.991334,Buzzd,9,"{'type': 'Point', 'coordinates': [-73.991334, ...",$4M,10,8430,253.00,4,4,4
54,Empresa tecnologica,USA,None,1985,40.731132,-73.991931,AOL,8000,"{'type': 'Point', 'coordinates': [-73.9919311,...",$1B,7,8282,248.53,4,4,4
40,Empresa de videojuegos,USA,None,2006,37.787092,-122.399972,Curse,58,"{'type': 'Point', 'coordinates': [-122.399972,...",$12M,50,3708,141.74,4,4,4
112,Empresa de videojuegos,USA,None,0,37.783898,-122.395234,Rupture,25,"{'type': 'Point', 'coordinates': [-122.395234,...",$3M,49,3624,139.21,4,4,4


In [ ]:
len(lista_clubs[5]['results'])

In [ ]:
(lista_clubs[1])

In [ ]:
asd
'''
    for i in range(len(lista_clubs[e]['name'])):
        print(i)
        for z in range(len(lista_clubs[e]['name'[z]])):
            print(z)
#lista_clubs[0]['results'][0]['geometry'])
len(asd)'''